In [2]:
import pandas as pd

In [5]:
data = pd.read_csv(r"C:\Users\zmitr\Desktop\Amazon-Sales-Analytics-for-Product-Profitability-Optimization\data\Amazon_Data_cleaned.csv")

In [7]:
data["date"] = pd.to_datetime(data["date"])

C:\Users\zmitr\AppData\Local\Temp\ipykernel_23180\1194260718.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["date"] = pd.to_datetime(data["date"])


In [8]:
dim_date = (
    data[["date"]]
    .drop_duplicates()
    .assign(
        date_id=lambda x: x.index + 1,
        day=lambda x: x["date"].dt.day,
        month=lambda x: x["date"].dt.month,
        year=lambda x: x["date"].dt.year
    )
)


In [9]:
dim_product = (
    data[["style","sku","category","size","asin"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_product["product_id"] = dim_product.index + 1


In [10]:
dim_location = (
    data[["ship_city","ship_state","ship_postal_code"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_location["location_id"] = dim_location.index + 1


In [11]:
dim_fulfilment = (
    data[["fulfilment","sales_channel","ship_service_level","courier_status","b2b"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_fulfilment["fulfilment_id"] = dim_fulfilment.index + 1


In [12]:
dim_status = (
    data[["status","status_clean"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_status["status_id"] = dim_status.index + 1


In [13]:
fact_orders = data.merge(dim_date, on="date") \
    .merge(dim_product, on=["style","sku","category","size","asin"]) \
    .merge(dim_location, on=["ship_city","ship_state","ship_postal_code"]) \
    .merge(dim_fulfilment, on=["fulfilment","sales_channel","ship_service_level","courier_status","b2b"]) \
    .merge(dim_status, on=["status","status_clean"])

fact_orders = fact_orders[[
    "order_id",
    "date_id",
    "product_id",
    "location_id",
    "fulfilment_id",
    "status_id",
    "qty",
    "amount"
]]


In [14]:
dim_date.to_csv("dim_date.csv", index=False)
dim_product.to_csv("dim_product.csv", index=False)
dim_location.to_csv("dim_location.csv", index=False)
dim_fulfilment.to_csv("dim_fulfilment.csv", index=False)
dim_status.to_csv("dim_status.csv", index=False)
fact_orders.to_csv("fact_orders.csv", index=False)

In [18]:

df = pd.read_csv(r"C:\Users\zmitr\Desktop\Amazon-Sales-Analytics-for-Product-Profitability-Optimization\data\Amazon_Data_cleaned.csv")  # replace with your file name


df['date'] = pd.to_datetime(df['date'])
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df['b2b'] = df['b2b'].fillna('B2C')

sales_summary = df.groupby(['sku', 'category', 'fulfilment'])['amount'].sum().reset_index().sort_values('amount', ascending=False)
print("=== Total Sales Amount per SKU, Category, Fulfillment ===")
print(sales_summary.head(10))


qty_summary = df.groupby(['sku', 'size', 'style'])['qty'].sum().reset_index().sort_values('qty', ascending=False)
print("\n=== Quantity Sold per SKU, Size, Style ===")
print(qty_summary.head(10))


aov = df.groupby('order_id')['amount'].sum().mean()
print("\n=== Average Order Value (AOV) ===")
print(aov)


revenue_category = df.groupby('category')['amount'].sum().reset_index().sort_values('amount', ascending=False)
print("\n=== Revenue per Category ===")
print(revenue_category)

revenue_sku = df.groupby('sku')['amount'].sum().reset_index().sort_values('amount', ascending=False)
print("\n=== Revenue per SKU ===")
print(revenue_sku.head(10))


total_orders = len(df)
on_time_orders = len(df[df['courier_status'].str.lower() == 'delivered'])
fulfillment_efficiency = on_time_orders / total_orders * 100
print("\n=== Fulfillment Efficiency (%) ===")
print(fulfillment_efficiency)


b2b_summary = df.groupby('b2b')['amount'].sum().reset_index()
print("\n=== B2B vs B2C Revenue ===")
print(b2b_summary)


qty_vs_revenue = df.groupby('sku').agg({'qty':'sum', 'amount':'sum'}).reset_index()
qty_vs_revenue['qty_per_amount'] = qty_vs_revenue['qty'] / qty_vs_revenue['amount']
high_qty_low_rev = qty_vs_revenue.sort_values('qty_per_amount', ascending=False)
print("\n=== SKUs with highest quantity but lowest revenue ===")
print(high_qty_low_rev.head(10))


daily_sales = df.groupby('date')['amount'].sum().reset_index()
print("\n=== Daily Sales ===")
print(daily_sales.head())


df['month'] = df['date'].dt.to_period('M')
monthly_sales = df.groupby('month')['amount'].sum().reset_index()
monthly_sales['MoM_Growth_%'] = monthly_sales['amount'].pct_change() * 100
print("\n=== Monthly Sales with MoM Growth ===")
print(monthly_sales)


df['quarter'] = df['date'].dt.to_period('Q')
quarterly_sales = df.groupby('quarter')['amount'].sum().reset_index()
print("\n=== Quarterly Sales ===")
print(quarterly_sales)


courier_issues = df.groupby('courier_status')['amount'].sum().reset_index().sort_values('amount', ascending=False)
print("\n=== Revenue per Courier Status ===")
print(courier_issues)


avg_revenue_sku = df.groupby(['category', 'sku'])['amount'].mean().reset_index().sort_values('amount', ascending=False)
print("\n=== Average Revenue per SKU ===")
print(avg_revenue_sku.head(10))


popular_sizes = df.groupby('size')['qty'].sum().reset_index().sort_values('qty', ascending=False)
popular_styles = df.groupby('style')['qty'].sum().reset_index().sort_values('qty', ascending=False)
print("\n=== Most Frequently Sold Sizes ===")
print(popular_sizes.head(10))
print("\n=== Most Frequently Sold Styles ===")
print(popular_styles.head(10))



C:\Users\zmitr\AppData\Local\Temp\ipykernel_23180\436540375.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


=== Total Sales Amount per SKU, Category, Fulfillment ===
                  sku       category fulfilment         amount
1949      J0230-SKD-M            Set     Amazon  482460.548873
1951      J0230-SKD-S            Set     Amazon  409032.055285
7033     JNE3797-KR-L  Western Dress   Merchant  399500.223119
7035     JNE3797-KR-M  Western Dress   Merchant  366278.119531
9845   SET268-KR-NP-S            Set     Amazon  287819.672269
9847  SET268-KR-NP-XL            Set     Amazon  287065.151040
1947      J0230-SKD-L            Set     Amazon  275498.397833
7037     JNE3797-KR-S  Western Dress   Merchant  272103.844436
9841   SET268-KR-NP-L            Set     Amazon  263036.712738
9184   SET183-KR-DH-M            Set   Merchant  255261.007643

=== Quantity Sold per SKU, Size, Style ===
                  sku size    style  qty
4548     JNE3797-KR-L    L  JNE3797  661
4549     JNE3797-KR-M    M  JNE3797  560
4550     JNE3797-KR-S    S  JNE3797  503
2746     JNE3405-KR-L    L  JNE3405  485
